# Instrument Classifier Demo

ToC goes here.

In [ ]:
# Global imports
from __future__ import print_function
import numpy as np
import theano
import theano.tensor as T

## I. Building a Deep Classifier
As we will see momentarily, building a deep network in Python can be quite straightforward with the help of Theano. The process consists of four stages, outlined below:

### 1. Define the Network’s Architecture
Despite the excitement circling the notion of “deep learning,” a deep network is simply a non-linear function that maps inputs to outputs. And, like any other function, it has some general form and is specified by its coefficients, e.g. a parabola. The advantage inherent to deep learning is that an arbitrarily complex equation can be chosen or designed, and the coefficients that best model this relationship can be found through numerical optimization.
Restating for convenience, Theano is a mathematical expression library, offering symbolic differentiation, the ability to compile symbolic functions into C-code, and tight integration with NVIDIA GPUs for parallelization. Of those three attributes, the first is of particular interest here. In the spirit of simplicity, Theano offers two basic kinds of symbolic datatypes: variables (vectors, matrices, tensors, etc), and shared parameters.

In a Theano expression, variables serve as placeholders for independent data (i.e. input variables) that will be pushed through the network. Therefore, it is necessary to indicate what the expected number of dimensions for this data by creating the right type, e.g. vector versus 3d-tensor. By and large, you will only ever need to explicitly declare variables for the inputs of a network, and will therefore have a very clear idea what your data looks like.
Conversely, shared parameters (known simply in Theano as a “shared” type) are used to represent the parameters of a model; it may help to think of shared types as coefficients. Shared parameters offer a few key benefits that should be addressed specifically:

* Shared parameters are persistent; you will not need to provide shared types as inputs to a function, but you may want to keep a handle to them.
* You can differentiate with respect to shared types (addressed here).
* Shared types can be updated implicitly during a function call (addressed here).

For the time being, it is sufficient to consider only the first point. As opposed to variables, shared types take and store real numerical values. This is a crucial distinction to make, and we call special attention to it now: a shared type owns persistent numerical values, which are allowed to change during program execution. In this sense, shared types must be initialized with some numerical value.

Having discussed the prerequisite Theano concepts, we can review a sample network definition to see these principles in practice:

In [ ]:
# ----------------------------------------------------
# Step 1. Build the network
# ----------------------------------------------------
x_input = T.matrix('input')

# Define layer shapes -- (n_in, n_out)
l0_dim = (120, 256)
l1_dim = (256, 256)
l2_dim = (256, 10)

# Build-in the standardization methods.
mu_obs = theano.shared(np.zeros(l0_dim[:1]), name='mu')
sigma_obs = theano.shared(np.ones(l0_dim[:1]), name='sigma')
x_input -= mu_obs.dimshuffle('x', 0)
x_input /= sigma_obs.dimshuffle('x', 0)

# Layer 0
weights0 = theano.shared(np.random.normal(scale=0.01, size=l0_dim),
                         name='weights0')
bias0 = theano.shared(np.zeros(l0_dim[1]), name='bias0')
z_out0 = hwr(T.dot(x_input, weights0) + bias0)

# Layer 1
weights1 = theano.shared(np.random.normal(scale=0.01, size=l1_dim),
                         name='weights1')
bias1 = theano.shared(np.zeros(l1_dim[1]), name='bias1')
z_out1 = hwr(T.dot(z_out0, weights1) + bias1)

# Layer 2 - Classifier Layer
weights2 = theano.shared(np.random.normal(scale=0.01, size=l2_dim),
                         name='weights2')
bias2 = theano.shared(np.zeros(l2_dim[1]), name='bias2')
z_output = T.nnet.softmax(T.dot(z_out1, weights2) + bias2)

First, we see the input to the network, a matrix, is declared in Line 157. This is consistent with the instrument data, which consists of Constant-Q vectors. The input data type is a matrix because we will provide batches of vectors, i.e. matrices, to the network.

Lines 12:15 create two shared parameters, taking the names mu and sigma, used to standardize the input data. For now, we initialize the means to a vector of zeros and the standard deviations to a vector of ones, effectively by-passing this operation. Later we will see how these parameters are updated with statistics computed over the training set, and the significance of naming these parameters with useful labels.

Lines 17:21 show the definition of the first layer of this network, an affine (fully-connected) transform. Here, a weight matrix and a vector of biases are initialized with normally distributed values and zeros, respectively. Note that these shared types are also given unique names, where the layer index is clearly indicated in the label. The output of the first layer, z_out0, is computed as the dot-product of the input x_input with the weight matrix weights0, followed by an additive bias bias0 and point-wise half-wave rectification (written elsewhere in the provided code).

Finally, the other two layers are almost identical to the first. The sole difference is the application of the softmax operation in Line 33. This enforces the output of the network to sum to 1 for a single datapoint. Further note that the output of this function, effectively a multiple observation posterior, will also be a matrix.

### 2. Define a Loss Function
Now that we have defined our classification network, we need to establish some means of measuring how well the current model fits our data. In the language of neural networks, this is known as a loss function, or simply the loss. For more information, we refer the curious reader to this tutorial on [Energy-based Learning](http://www.cs.nyu.edu/~sumit/research/assets/ebmtutorial.pdf).

For a network that aims to classify an observation as one of a predefined set of classes, we would like to make the loss of the “correct” class smaller than the loss of all other classes. Since we have defined a network that produces a posterior, we can define the loss as the negative log-likelihood of this output:

In [ ]:
# ----------------------------------------------------
# Step 2. Define a loss function
# ----------------------------------------------------
y_target = T.ivector('y_target')
observation_index = T.arange(y_target.shape[0], dtype='int32')
scalar_loss = T.mean(-T.log(z_output)[observation_index, y_target])
accuracy = T.mean(T.eq(T.argmax(z_output, axis=1), y_target))

To implement this in Theano, we first need to declare an integer vector input for the correct classes of the data processed by the network (Line 4). Similarly, we also need to create a range vector (Line 5) that spans from [0:batch_size]. These two vectors are used to index the posterior and select the values of the right answers (Line 6). Note that this produces a symbolic scalar loss over a batch of data, where the loss of multiple observations are averaged together.

For completeness, we can also define a symbolic scalar accuracy value (Line 7). This is defined here by comparing the predicted classes (the argmax of the posterior, z_output) with the given correct classes (y_target). A prediction is correct when these two vectors are equal, and the accuracy is computed as the mean value over these observations.

### 3. Compute Parameter Update Rules
Importantly, when the fitness of a model can be calculated as a scalar output of a differentiable function, gradient optimization can be used to find, or learn, better values for the parameters. The intuition behind this method is actually a simple one; move the parameter values in tiny steps toward more “correct” answers:

In [ ]:
# ----------------------------------------------------
# Step 3. Compute Update rules
# ----------------------------------------------------
eta = T.scalar(name="learning_rate")
updates = OrderedDict()
network_params = OrderedDict()
for param in [weights0, bias0, weights1, bias1, weights2, bias2]:
    # Save each parameter for returning later.
    network_params[param.name] = param
    # Compute the gradient with respect to each parameter.
    gparam = T.grad(scalar_loss, param)
    # Now, save the update rule for each parameter.
    updates[param] = param - eta * gparam

Like before, the first step is to create a scalar input, eta, to control the step size, or learning rate, at every update iteration (Line 4). Then, for each parameter in the network we’d like to update, we compute the gradient with respect to it (Line 11). Then, we can define an update rule for this parameter, as its difference with its gradient, scaled by eta (Line 13).

This final aspect of Theano --update functions-- is very useful, if somewhat opaque, so we draw attention to it here. In Python / Theano, it is possible to define an update function for shared types, stored in an Ordered dictionary and keyed by a specific instance. As we will see in the next stage, these update functions can be implicitly invoked for each shared parameter, making the learning process significantly easier and effectively transparent.

### 4. Compile Theano Functions
As mentioned at the start of this section, Theano enables mathematical expressions to be compiled to C-code. Not only does this accelerate the run-time of computation, but it also enables shared parameters to be updated behind the scenes.

In [ ]:
# ----------------------------------------------------
# Step 4. Compile wicked fast theano functions!
# ----------------------------------------------------
objective_fx = theano.function(inputs=[x_input, y_target, eta],
                               outputs=100*(1.0 - accuracy),
                               updates=updates,
                               allow_input_downcast=True)

prediction_fx = theano.function(inputs=[x_input],
                                outputs=z_output,
                                allow_input_downcast=True)

# Add mu and sigma variables now, as we don't want to update them
# during training.
network_params.update({mu_obs.name: mu_obs,
                       sigma_obs.name: sigma_obs})

In practice, this proceeds as follows: all symbolic variables necessary to compute some output, which can be nearly anything, are provided as a list of inputs; this output is declared as the output; any update rules, as a dictionary of functions keyed by shared parameter, are provided to the updates argument. As an implementation detail, we allow downcasting from float64 to float32. The result is a function that can be called like any standard Python method.
Before proceeding, compare the two functions, objective_fx and prediction_fx. 

The first takes three arguments: x_input, the data to transform, y_target, the posterior indices corresponding to the correct classes, and eta, the learning rate. Update rules are defined for this function, so the numerical values for the parameters listed previously will change at every iteration eta > 0. Conversely, the second function takes only a single argument, x_input, and has no defined update rules. This function is used solely for inference, or computing the posterior for each row in the input matrix.

## II. Dataset
To facilitate the exploration of deep learning with musically relevant problems, we provide a precomputed dataset of constant-Q coefficients for 10 different instrument classes, sampled from the University of Iowa Music Instrument Samples.

uiowa_instrument_dataset.tgz

Be aware that this is, strictly speaking, a toy dataset and its use in “real” research is discouraged. However, it will more than serve our purposes to demonstrate how to implement deep network classifier. 

For more information regarding what exactly this data represents, please consult the README contained in the archive.

### Shuffling the Data
It is yet another advantage of deep learning algorithms that they are able to synthesize arbitrarily large datasets during training. This is achieved by computing gradients over small data batches and updating the parameters of the network accordingly; this is referred to as mini-batch optimization.

Though there is some theory developing around guiding heuristics for populating a mini-batch at a given iteration [see [Bengio2009]()], randomly sampling the training set has shown to work reasonably well in practice. To these ends, we write a generator that returns batches of data and the corresponding labels forever.

In [ ]:
def data_shuffler(data, labels, batch_size=100):
    """Data shuffler for training online algorithms with mini-batches.

    Parameters
    ----------
    data : np.ndarray
        Data observations with shape (n_samples, dim0, dim1, ... dimN).
    labels : np.ndarray
        Integer targets corresponding to the data (x).

    Yields
    ------
    x_m : np.ndarray
        Data with shape (batch_size, dim0, dim1, ... dimN).
    y_m : np.ndarray
        Targets corresponding to the samples in x_m.
    """
    num_samples = len(data)
    sample_idx = np.arange(num_samples, dtype=np.int32)
    read_ptr = num_samples

    while True:
        x_m, y_m = [], []
        while len(x_m) < batch_size:
            if read_ptr >= num_samples:
                np.random.shuffle(sample_idx)
                read_ptr = 0
            x_m.append(data[sample_idx[read_ptr]])
            y_m.append(labels[sample_idx[read_ptr]])
            read_ptr += 1
        yield np.array(x_m), np.array(y_m)

Here, this function consumes the full matrix of feature observations and corresponding vector of labels, and returns (at each call to next()) a randomly sampled subset of batch_size points. Notice that this method will produce infinite permutations of the dataset with minimal computational overhead, as the order of the data is only shuffled after a complete epoch (Lines 57:59).


### Stepping through Data

Similar to the goals of a data shuffler, there may be instances where a collection of data is too large to transform all at once. We can again use a generator to step through this data for us, making it very easy to iterate over non-overlapping slices of the data.

In [ ]:
def data_stepper(data, labels, batch_size=100):
    """Generator for stepping through a dataset in batches.

    Parameters
    ----------
    data : np.ndarray
        Data observations with shape (n_samples, dim0, dim1, ... dimN).
    labels : np.ndarray
        Integer targets corresponding to the data (x).

    Yields
    ------
    x_m : np.ndarray
        Data with shape (batch_size, dim0, dim1, ... dimN).
    y_m : np.ndarray
        Targets corresponding to the samples in x_m.
    """
    num_samples = len(data)
    read_ptr = 0
    x_m, y_m = [], []
    while read_ptr < num_samples:
        x_m.append(data[read_ptr])
        y_m.append(labels[read_ptr])
        read_ptr += 1
        if len(x_m) == batch_size:
            yield np.array(x_m), np.array(y_m)
            x_m, y_m = [], []

    if len(x_m) > 0:
        yield np.array(x_m), np.array(y_m)

Shortly (in Inference), we will see how to use an instance of this generator and list comprehension to compactly transform large collections of data without clogging up system resources.

## III. Training
Now that the network training and inference have been compiled to Theano functions, we can use the data shuffling strategy discussed above to actually train the model. This is an iterative process that often runs for a large number of iterations. As we will see, our experiments show 40-50k iterations to be sufficient; more than this runs the risk of overfitting.

In [ ]:
def train_network(objective_fx, shuffler, learning_rate, num_iterations,
                  print_frequency=100):
    """Run the training process for some number of iterations.

    Parameters
    ----------
    objective_fx : compiled theano function
        First function returned by build network; updates the parameters as
        data is passed to it.
    shuffler : generator
        Data source with a next() method, returning a two-element tuple (x,y).
    learning_rate : scalar
        Update rate for each gradient step.
    num_iterations : int
        Number of update iterations to run.
    print_frequency : int
        Number of iterations between printing information to the console.

    Returns
    -------
    training_error : np.ndarray
        Vector of training loss values over iterations.
    """
    training_error = np.zeros(num_iterations)
    n_iter = 0
    try:
        while n_iter < num_iterations:
            x_m, y_m = shuffler.next()
            training_error[n_iter] = objective_fx(x_m, y_m, learning_rate)
            if (n_iter % print_frequency) == 0:
                print("[{0}]\t Iter: {1} \tTraining Error: {2}"
                      "".format(time.asctime(), n_iter, training_error[n_iter]))
            n_iter += 1
            
    except KeyboardInterrupt:
        print "Stopping Early."

    return training_error[:n_iter]

The important parts of this code block are found in Lines 256:257. Training simply consists of sampling the shuffler by calling next(), which yields a batch of data, x_m, and corresponding labels, y_m, and passing them to the objective function along with a learning rate. This call returns the average classification error over the batch, written to a pre-allocated array, and after potentially printing its progress to the console, the loop repeats.

As will be discussed in the next section, this while-statement is wrapped in a try-except block that gracefully exits when the user triggers a keyboard interrupt, ctrl+C. This is provided as a manual form of early stopping, but in practice you’ll probably want to develop a heuristic to do this automatically.

In [ ]:
# Load data
# x, y = np.load()
# data = shuffler(x, y, 100)

# Actually train the model.
training_error = train_network(objective_fx, data, 0.1, 10000, -1)

# Plot a graph

## IV. Inference
After training the network to completion, we can investigate how this model generalizes to the holdout test set. Using the data stepper discussed above makes this really easy:

In [ ]:
test_data = data_stepper(
    np.load(args.test_data_file), np.load(args.test_label_file), 500)
correct_predictions = [np.equal(prediction_fx(x_m).argmax(axis=1),
                                y_m) for x_m, y_m in test_data]
test_error = 1.0 - np.concatenate(correct_predictions).mean()
print("Test Error: {0}".format(100*test_error))

Here, we take advantage of list comprehension to apply the prediction function prediction_fx to each batch of data in the test set. At each iteration, the argmax of the posterior over x_m is computed and compared with the correct classes in y_m, returning a list of booleans indicating whether the observation was predicted correctly or not. From this, we can take the mean of the overall accuracy, and subtract it from 1 to get the total error.

As a point of reference, running this code with a learning rate of 0.025 for 50k iterations (run-time of 6 minutes) resulted in a training error below 5%, and a test error of 6.23%. Using sklearn’s SVM classifier (SVC) with a linear kernel obtains a training error of 15.23% and a test error of 19.07%.

## V. Next Steps
At this point, we’ve covered enough of the provided source code that you should be able to comfortably follow the program execution of training a deep network. There are a several exercises you may want to explore with this dataset:
* How might you implement automatic early stopping?
* What are the effects of batch size and learning rate? Or a variable learning rate?
* How does performance change as a function of layer width (size of each transform) or depth (number of layers)?
* Do different non-linearities change affect performance? Or initial parameters?